In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import pandas as pd

from helper import get_penmap

Using TensorFlow backend.


In [2]:
def compute_dist(param1,param2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                           
                        except:
                            
                            pass
     
    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
    return same_pens,diff_pens

def pen_distribution(features,a,pen_out):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        if(k in a[pen_out]):
            for i in ['1','2']: 
                for n in ['1','2']:
                    for j in ['1','2','3','4','5','6','7','8'] :
                        for m in ['1','2','3','4','5','6','7','8'] :
                            try:
                                if(i==n and j==m):
                                    continue    
                                elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                    same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                    diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                                 
                            except:

                                pass

    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
    return same_pens,diff_pens



In [3]:
def distance_btw_distributions(feature,a):
    d = []
    same_pens,diff_pens = distance_distribution(feature)
    D = dist(same_pens,diff_pens,1465,1456)
    for j in range(1,15):
        same_pens_i,diff_pens_i= pen_distribution(feature,a,j)
        d1 = len(same_pens_i)/2930*dist(same_pens_i,same_pens,len(same_pens_i),1465)
        d2 = len(diff_pens_i)/2912*dist(diff_pens_i,diff_pens,len(diff_pens_i),1456)
        d.append(d1+d2)
        
    return D-sum(d)

In [4]:
def load_parameters(path,n_param):
    hist_Y = {}
    hist_Cb = {}
    hist_Cr = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][0:n_param] 
                    hist_Cb['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n_param:2*n_param]
                    hist_Cr['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][2*n_param:3*n_param]
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cb['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cr['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    
                except:
                    break
    return hist_Y,hist_Cb,hist_Cr

In [5]:
def separate(h1,f):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.array([h1['c'+str(k)+'_'+i+'_'+j][f]])
                except:
                    break
    return hist

def concat2(h1,h2):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist

def concat3(h1,h2,h3):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist


In [6]:
a = get_penmap()

In [7]:
path = '/home/mohit/Desktop/featuresUpdated/GLCM_mean_ptp.csv'
y,cb,cr = load_parameters(path,26)

In [13]:
fe = {}

for i in range(1,27):
    fe[i]    = separate(y,i-1)
    fe[i+26] = separate(cb,i-1)
    fe[i+52] = separate(cr,i-1)

In [14]:

# min max normalising every feature
for i in range(1,79):
    mi = np.min([ v for v in fe[i].values() ])
    mx = np.max([ v for v in fe[i].values() ])
    for key in fe[i].keys():
        fe[i][key]-=mi
        fe[i][key]/=(mx-mi)    


In [16]:
Distances = {}

for i in range(1,79):
    Distances[i] = distance_btw_distributions(fe[i],a)
        
sorted_by_value = sorted(Distances.items(), key=lambda kv: kv[1],reverse =True)
sorted_by_value[0][0],sorted_by_value[0][1]

(6, 17.7170148251883)

### after comparing all features, we found that fe[6] i.e 6_y is the best and will be the starting point of our forward selection technique.

In [17]:
feycb = {}
feycr  ={}
feycbcr = {}
for i in range(1,27):
    feycb[i] = concat2(fe[i],fe[i+26])
    feycr[i] = concat2(fe[i],fe[i+52])
    feycbcr[i] = concat3(fe[i],fe[i+26],fe[i+52])

In [18]:
# fmax will contain the selected features. dmax will contain the increment by adding the ith feature. 
# Fmax will contain a dict with all selected features concatenated together.

fmax = [6]
dmax = [17.717014825188294]
Fmax = fe[6]
Fmaxycb = feycb[6]
Fmaxycr = feycr[6]
Fmaxycbcr = feycbcr[6]

In [19]:
Dmax = sum(dmax)
f = range(1,27)

In [20]:
while(1):
    Distancesy = {}
    
    for i in set(f)-set(fmax):
        Distancesy[i] = distance_btw_distributions(concat2(Fmax,fe[i]),a) - Dmax
        
    sorted_by_value = sorted(Distancesy.items(), key=lambda kv: kv[1],reverse =True)
    dd = sorted_by_value[0][1]
    
    Distancesycb = distance_btw_distributions(Fmaxycb,a)-Dmax
        
    Distancesycr = distance_btw_distributions(Fmaxycr,a)-Dmax
    
    print(sorted_by_value[0][0],dd,Distancesycb,Distancesycr)
    
    if(dd<Distancesycb or dd<Distancesycr or dd<0):
        if(max(Distancesycb,Distancesycr)>0):
            dmax.append(max(Distancesycb,Distancesycr))
            Dmax = sum(dmax)
        break            
                
    fmax.append(sorted_by_value[0][0])
    dmax.append(sorted_by_value[0][1])
    Fmax = concat2(fe[sorted_by_value[0][0]],Fmax)
    Dmax = sum(dmax)
    Fmaxycb = concat2(feycb[sorted_by_value[0][0]],Fmaxycb)
    Fmaxycr = concat2(feycr[sorted_by_value[0][0]],Fmaxycr)
    Fmaxycbcr = concat2(feycbcr[sorted_by_value[0][0]],Fmaxycbcr)

4 2.06538716347233 0.31525803954177434 0.04787547882671106
10 1.7759604500797188 3.115767459599592 -2.300686456121756


In [21]:
fmax,dmax,Dmax

([6, 4],
 [17.717014825188294, 2.06538716347233, 3.115767459599592],
 22.898169448260216)

### after applying our strategy, we found the following selection pattern. 

### 6y->4y -> 4 ycb

### so now we will apply forward selection on ycb channel with fstart = [6,4]

In [22]:
while(1):
    Distancesycb = {}
    
    for i in set(f)-set(fmax):
        Distancesycb[i] = distance_btw_distributions(concat2(Fmaxycb,feycb[i]),a)-Dmax
        
    sorted_by_value = sorted(Distancesycb.items(), key=lambda kv: kv[1],reverse =True)
    dd = sorted_by_value[0][1]
    
    Distancesycbcr = distance_btw_distributions(Fmaxycbcr,a)-Dmax
        
    print(sorted_by_value[0][0],dd,Distancesycbcr)
    if(dd<Distancesycbcr or dd<0):
        if(Distancesycbcr>0):
            dmax.append(Distancesycbcr)
            Dmax = sum(dmax)
        break
                
    fmax.append(sorted_by_value[0][0])
    dmax.append(sorted_by_value[0][1])
    Dmax = sum(dmax)
    Fmaxycb = concat2(feycb[sorted_by_value[0][0]],Fmaxycb)
    Fmaxycbcr = concat2(feycbcr[sorted_by_value[0][0]],Fmaxycbcr)

3 3.8985795244696035 -6.197168989074736
7 0.6707161910442672 -1.2616128697884186
16 0.7916820425626625 -1.1745311603316253
10 0.43570616510903903 0.5591314222349837


In [23]:
fmax,dmax,Dmax


([6, 4, 3, 7, 16],
 [17.717014825188294,
  2.06538716347233,
  3.115767459599592,
  3.8985795244696035,
  0.6707161910442672,
  0.7916820425626625,
  0.5591314222349837],
 28.818278628571733)

### sequence of selction

###  6y  -> 4y -> 4ycb -> 3ycb -> 7ycb -> 16ycb -> 16ycbcr

### so we have to apply forward selection on ycbcr channel with fstart =[6,4,3,7,16]

In [24]:
while(1):
    Distancesycbcr = {}
    for i in set(f)-set(fmax):
        Distancesycbcr[i] = distance_btw_distributions(concat2(Fmaxycbcr,feycbcr[i]),a)-Dmax
        
    sorted_by_value = sorted(Distancesycbcr.items(), key=lambda kv: kv[1],reverse =True)
    dd = sorted_by_value[0][1]
        
    print(sorted_by_value[0][0],dd)
    if(dd<0):
        break
                
    fmax.append(sorted_by_value[0][0])
    dmax.append(sorted_by_value[0][1])
    Dmax = sum(dmax)
    Fmaxycbcr = concat2(feycbcr[sorted_by_value[0][0]],Fmaxycbcr)

10 0.17035093482024166
25 0.26912142723886134
12 0.1182681293589205
2 -0.3113542759276413


In [27]:
fmax.sort()
fmax

[3, 4, 6, 7, 10, 12, 16, 25]

### So, our feature subset is [3, 4, 6, 7, 10, 12, 16, 25]  in all 3 channels with a distance of

In [28]:
Dmax

29.376019119989756

In [30]:
# Lets save original features (not normalised) !
f = {}
for i in [3, 4, 6, 7, 10, 12, 16, 25]:
    f[i] = concat3(separate(y,i-1),separate(cb,i-1),separate(cr,i-1))

In [40]:
best = f[3]
for i in [4, 6, 7, 10, 12, 16, 25]:
    best = concat2(best,f[i])

In [41]:
best = pd.DataFrame(best)
best

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,0.272580,0.343392,0.374409,0.304421,0.214561,0.249565,0.249783,0.344433,0.342782,0.297526,...,0.151027,0.387566,0.403251,0.499124,0.508989,0.500553,0.411988,0.501667,0.492740,0.424042
1,0.712217,0.734085,0.700457,0.697822,0.164977,0.150165,0.131484,0.310955,0.308627,0.239849,...,0.163037,0.695568,0.728904,0.686154,0.696058,0.667423,0.701632,0.678424,0.763586,0.789137
2,0.298065,0.308474,0.312066,0.303538,0.632220,0.607482,0.593275,0.161111,0.164981,0.110454,...,0.607635,0.769401,0.805356,0.806033,0.824035,0.818250,0.818939,0.810804,0.818478,0.821633
3,178.902406,237.959759,296.593262,214.237673,242.034295,303.703468,308.347796,369.493873,362.386410,317.556128,...,185.160851,249.247097,305.348805,558.952004,532.089659,583.254714,387.462956,507.237644,568.566861,413.714900
4,2.591589,3.237982,3.713286,3.471541,0.234606,0.200418,0.171750,0.232800,0.328055,0.279325,...,0.201520,4.672811,5.872723,5.995486,6.429594,6.719794,7.869853,8.749872,9.463638,17.780395
5,0.223675,0.306863,0.302403,0.384304,4.770049,3.716620,3.406547,0.105045,0.165687,0.098417,...,4.035973,117.837300,154.013454,156.856839,164.395254,182.414403,220.149930,243.278295,176.483323,292.092323
6,137.658997,144.018704,129.503144,140.316276,82.041390,77.188933,74.303321,74.795613,71.817706,70.959082,...,75.847598,173.495098,152.852597,148.581624,144.923832,141.578143,148.027779,147.209341,121.201215,128.438125
7,257.066029,257.277432,257.653725,257.505181,256.072326,256.039748,256.029547,256.279257,256.329725,256.403301,...,256.062202,254.665335,254.440296,254.395904,254.425316,254.209042,253.957464,253.559709,254.078490,252.742171
8,255.693343,255.620497,255.517969,255.517570,257.851529,257.440097,257.307987,255.881086,255.889596,255.938329,...,257.706446,262.830393,264.139721,264.673620,264.686225,266.084360,267.923589,269.817363,264.072179,269.097300
9,1674.470344,1863.341474,2178.748660,1836.050487,2732.690115,3044.129653,3087.878565,2885.693794,2844.010720,2772.518665,...,2781.123747,1788.601158,2130.868731,3361.042080,3157.973145,3500.836655,2656.521356,3038.333079,3446.780292,2775.769078


In [42]:
best.to_csv('/home/mohit/Desktop/featuresUpdated/Glcm_Selectedfeatures.csv')

# remark:
I've calculated both, standard deviation and range, and compared the distances. to select the best among those im using majority voting . SD was voted 17 and PTP was voted 22 times.
So we're sticking to PTP